In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime as dt
from pandas_ta.candles import cdl_doji, cdl_pattern
import plotly.graph_objects as go


import os
import pickle
from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
import scripts.functions as fn 
from scripts.indicators import HeikinAshi, resample

from binance.client import Client

#Conexion con Binance sin API-KEY
client = Client()

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [27]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [28]:
klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
with open(klines_file, 'rb') as file:
    df = pickle.load(file)
    #df.set_index('datetime',inplace=True)

df = resample(df,(60*24))
    
#start = '2024-11-12'
#end = '2024-11-13'
#symbol = 'BTCUSDT'
#interval = '5m'
#real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
#real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
#utc_start = real_start+' 00:00:00'
#utc_end = real_end+' 23:59:59'
#df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)

#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


https://www.litefinance.org/es/blog/for-beginners/graficos-de-forex-en-linea-tipos-de-graficos-y-su-analisis/velas-heiken-ashi/

#DOJI
Se forma cuando los precios de apertura y cierre son casi iguales. En este punto, la fuerza de los alcistas y bajistas es igual, mostrando incertidumbre en el mercado. Cuando Doji aparece en la parte superior de una tendencia alcista, prepárese para abrir una posición corta y, cuando es en la parte inferior de una tendencia bajista, abra una posición larga. Para mayor fiabilidad, combine las señales de Doji con un oscilador, como el estocástico.

In [32]:

# Crear una copia para trabajar en Heikin-Ashi con el close promedio entre close anterior y actual
df = HeikinAshi(df)

df['ma_vol'] = df['volume'].rolling(window=100).mean()

df['cdl_doji'] = cdl_doji(df['HA_open'],df['HA_high'],df['HA_low'],df['HA_close'])
df['cdl_haramicross'] = cdl_doji(df['HA_open'],df['HA_high'],df['HA_low'],df['HA_close'],name='haramicross')
df['cdl_hammer'] = cdl_doji(df['HA_open'],df['HA_high'],df['HA_low'],df['HA_close'],name='hammer')

df['buy'] = np.where((df['cdl_hammer']>90) ,df['close']*.95,None)
#df['buy'] = np.where((df['cdl_hammer']>90) & (df['HA_side']>0) & (df['volume']>df['ma_vol']),df['close'],None)
#df['sell'] = np.where((df['cdl_hammer']>90) & (df['HA_side']<0) & (df['volume']>df['ma_vol']),df['close'],None)


In [33]:
df_to_show = df #[0:20]
#df_to_show['price'] = df_to_show['close']
df_to_show['close'] = df_to_show['HA_close']
df_to_show['open'] = df_to_show['HA_open']
df_to_show['high'] = df_to_show['HA_high']
df_to_show['low'] = df_to_show['HA_low']
indicators = [
     #{'col': 'HA_sl','name': 'HA_sl','color': 'red','row': 1, 'mode':'lines',},
     #{'col': 'HA_tp','name': 'HA_tp','color': 'green','row': 1, 'mode':'lines',},
     #{'col': 'doji','name': 'doji','color': 'red','row': 1, 'mode':'markers',},
     #{'col': 'st_high','name': 'ST High','color': 'green','row': 1, 'mode':'markers',},
     #   {'col': 'FL_0.0%', 'name': 'FL_0.0%', 'color': 'red', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold1', 'name': 'FL_gold1', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_gold2', 'name': 'FL_gold2', 'color': 'orange', 'row': 1, 'mode': 'lines', }, 
     #   {'col': 'FL_100.0%', 'name': 'FL_100.0%', 'color': 'green', 'row': 1, 'mode': 'lines', }, 
        ]

indicators_out = [
     #{'col': 'pred','name': 'pred','color': 'yellow','row': 1, 'mode':'lines',},
     #{'col': 'VOL_perc','name': 'Volume_SMA','color': 'green','row': 1, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       {'df':df,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()